In [ ]:
#센서데이터를 기반으로 Device 근처에 사람이 있는지 없는지 체크해보기

In [ ]:
import os # 파일 경로
import numpy as np #선형대수학
import pandas as pd # 데이터 처리 및 csv파일 입출력을 위해서 (e.g pd.read_csv)
import matplotlib.pyplot as plt # 그래프 시각화를 위해
import seaborn as sns #시각화


In [ ]:
cur_path = os.getcwd() #현재 작업중인 디렉터리 참조 이 디렉터리 내부에 csv파일이 존재하고 있음.
data_path = os.path.join(cur_path, 'iot_telemetry_data.csv')
print(data_path)

In [ ]:
data = pd.read_csv(data_path, engine='python')

## Task

* ML을 이용하여 사람이 Device 근처에 있는지 판단하기  


* 사람이 IoT 디바이스 근처에 있을때의 시간에 대한 레이블이 없기 때문에 비지도학습으로 제한 된다.  


* 비지도 학습은 정답에 대한 레이블이 없는 경우에 비슷한 데이터끼리 군집화를 하여 새로운 데이터에 대한 예측을 하는 방법이다.


* 3개의 다른 디바이스가 3곳의 다른 장소에 있다. 5~10초 간격으로 수집된 데이터들이다.


* 사람이 기록된 데이터 파라미터중 영향을 줄 수 있는 것들은 다양한 것이 있다.



- CO(일산화탄소) 농도가 사람 1명에 의해서 영향을 받을까? 아마도 센서 근처에서 사람이 호흡을 했다면 <strong>일산화탄소의 농도 감소</strong>에 기여했을것이다. 사람의 혈액에 있는 헤모글로빈은 일산화탄소와 결합력이 매우 높아서 (일산화탄소 중독의 원인이 바로 저 강한 결합력 -> 일산화탄소와 헤모글로빈이 결합하면 헤모글로빈의 산소 운반력이 줄어든다.


- 습도는 사람이 방 안에서 숨을 내쉴때 증가할 것이다.


- 빛은 사람이 빛 센서를 가렸다면 감소할 것이고, 사람이 불일 켰다면 아마 증가했을 것이다.


- LPG 농도는 아마 감소할것이다. 사람의 폐가 LPG를 필터하는 역할을 했을 것


- 센서 근처에서의 사람의 움직임은 아마 움직임 감지 센서를 작동시켰을 것이다. 그렇지만 사람이 근처에 있었다는것을 정확히 하려면 다른 모션센서나 진동 센서가 필요하다.


- 연기(smoke)의 레벨은 사람이 근처에 있었다면 감소했을 것이다. 사람의 폐가 필터링 역할을 하니까, 하지만 센서 앞에서 사람이 담배를 폈다면 증가했을 것


- 온도는 사람이 센서 근처에 있었다면 일시적으로 증가했을 것이다.

## Approach
- 의미있는 시간대에 대해서 데이터를 시각화 해보기(아마 낮에 사람이 더 활동을 많이 하니까 낮시간대로)


- 하루 주기의 패턴이 나타나는지 파악해보기 


- 세 다른 장소에서 특별히 눈에 띌만한 데이터의 차이점이 있는 시간대가 있는지 확인해보기


- 특정 시간대에 대해서 사람이 디바이스 근처에 있었다는것을 정의핳 임계치와, 신뢰구간을 찾아보자.

# 시간대를 찾아보자!

In [ ]:
# true / false로 되어있는 불리언 형태의 컬럼을 plotting을 위해서 정수형 데이터로 변환한다. (32bit 정수니까 4byte 정수)
data['light_int'] = data['light'].astype('int32')
data['motion_int'] = data['motion'].astype('int32')

In [ ]:
# 유닉스 타임스탬프를 날짜로 변환해보자
from datetime import datetime, timedelta
start = datetime(1970, 1, 1) #유닉스 타임 스탬프는 1970 1월 1일 00시 00분 00초를 기준으로 1초단위 진행
data['datetime'] = data.ts.apply(lambda x: start + timedelta(seconds=x))
data['string_time'] = data.datetime.apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# 각각의 디바이스에 대한 데이터를 분리한다.
data_device_gb = data.groupby('device')

In [ ]:
for i in data_device_gb:
    print(i[0])

In [ ]:
!pip install -q plotly

In [ ]:
# 데이터를 더욱 의미있는 시간축으로 더블클릭해서 볼 수 있게 그리기
cols = data.columns
unwanted_cols = set(['motion','ts', 'device', 'light', 'datetime', 'string_time'])

import plotly.express as px 

plt_idx = 0
for z in data_device_gb:
    fig = px.line(log_y = True, title = z[0])
    for i, j in enumerate(cols):
       # print(i)
        if j in unwanted_cols:
            continue
        else:
            fig.add_scatter(x=z[1].iloc[:,-1], y=z[1].iloc[:,i], mode='lines')
            #print(i, j)
            fig.data[plt_idx].name = j
            plt_idx += 1

    fig.show()
    fig.data = []
    plt_idx = 0

![t1](images/time1.png)
![t2](images/time2.png)
![t3](images/time3.png)

# 위 Plot에서 볼 수 있는 것들
- CO, LPG 그리고 smoke 레벨이 (공기의 질과 관련된 파라미터들) 각 디바이스간에 상관관계가 있었다.
- 온도에 급격한 변화가 기록되었다(센서 오류 ?_?)
- 습도와 모션에 튀는 레코드들이 있었다.


In [ ]:
subset = set(['smoke', 'humidity', 'temp'])
f, axes = plt.subplots(1, 3, figsize=(30, 10))

for i, j in enumerate(subset):
    sns.boxplot( y=data[j], x="device", data=data, hue = 'device', orient = 'v', ax=axes[i])

![box](images/box.png)

# 서로 다른 세 장소에있는 세 센서에 다른 점이 있었나?

- 00:를 00이라고하고, 1c:를 1c, b8:를 b8 디바이스라고 하자.

    - 모션 spikes는 모션이 없는 간격에서 산재해있다. b8과 c1은 00보다 더 큰 motion spike를 보여줬다.
    - 1c 다바이스는 계속해서 밝았지만 나머지 두개는 밝고 어두운 간격들을 갖고있다.
    - 00은 다른 디바이스들보다 매우 나쁜 공기의 질(smoke, lpg 등등 지표를 보았을때)을 갖고있었다.
    - 박스 플롯을 통해서 세 디바이스가 완전히 구별되는 다른 공간에 있음을 확인 할 수 있다.
    
    
    1. 공기의 오염도는 b8에서 제일 높았고, 그 다음으로 1c, 00 순이였다. 하지만 박스 플롯을 확인해보면 00 디바이스가 제일 spike(튀는 값)은 많았다.
    2. 세 디바이스는 약간 다른 평균 온도를 보여줬다(20~30도 사이에서) 00과 1c는 매우 밑으로 튀는 값이 많았다.
    3. 세 디바이스는 서로 다른 습도 레벨을 보여줬다. 50~75%사이에서. 세 디바이스는 이상치 이상으로 오르거나 내리는 부분이 많았다. 1c는 0~65%까지 매우 많은 이상치가 있었다.
        
        
# 데일리 패턴이 있었을까?
- Smoke를 대상으로 시간대별 트렌드 그레프를 계산해보자



In [ ]:
!pip install -q fbprophet

In [ ]:
data['ds'] = data['datetime']
data['y'] = data['smoke']
data_device_gb = data.groupby('device')

In [ ]:
# 그룹으로 나뉘어진 데이터로 부터 데이터프레임 사전형 자료를 생성
df_dict = {}
for i, j in enumerate(data_device_gb):
    df_dict[i] = j[1]

In [ ]:
# 밑에 코드들은 실행하는데 매우 오래 걸린다
from fbprophet import Prophet

In [ ]:
m = Prophet() #예측 모델

prophet_dict = {}
for i in df_dict:
    prophet_dict[i] = m.fit(df_dict[i][['ds','y']])
    m = Prophet()

In [ ]:
future_dict = {} 
for i in prophet_dict:
    m = prophet_dict[i]
    future_dict[i] = m.make_future_dataframe(periods=0, freq='H')
# future = m.make_future_dataframe(periods=0, freq='H')

In [ ]:
fcst_dict = {}
for i in future_dict:
    m = prophet_dict[i]
    fcst_dict[i] = m.predict(future_dict[i])
# fcst = m.predict(future)

In [ ]:
for i in fcst_dict:
    m = prophet_dict[i]
    fig = m.plot_components(fcst_dict[i])
    ax = fig.gca()
    ax.set_title("Smoke - Device {}".format(i+1), size=16, loc = 'right')
#fig = m.plot_components(fcst)

![1](images/1.png)
![2](images/2.png)
![3](images/3.png)

## 결과

- 위의 fbprophet을 이용한 ML결과를 보면, smoke 레벨의 트렌드를 확인 할 수 있는데 6~8시 사이에는 떨어지다가 그 이후 매 자정마다 세 디바이스의 스모크 레벨이 모두 상승하는 것을 볼 수 있다. 세 디바이스의 세 지점 모두에서 나타나는 트렌드이다.

# 비지도 학습의 결과

- 사람이 센서 주변에 있을때의 센서 데이터를 추가 확보 해야 할것이고, 사람이 주기적으로 나타나는지, 아니면 랜덤하게 나타나는지에 대한 정보가 없어서 정확한 유추는 힘들다.
- 다만 우리가 유추할 수 있는 부분은 사람이 나타난다면
    - 모션 데이터의 spike
    - 빛 데이터의 spike
    - 공기 오염의 spike
    - 온도와 습도의 spike

- 어렵다..